In [13]:
import pandas as pd
import numpy as np

# load raw training data
train_raw = pd.read_csv('data/train.csv')

In [14]:
# drop irrelivant features
cols_to_drop = ['id', 'year', 'zip.code']
train_raw = train_raw.drop(cols_to_drop, axis=1)

print("\n--DISPLAYING DATAFRAME INFO--\n")
print(train_raw.info(), "\n")


--DISPLAYING DATAFRAME INFO--

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 15 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   cancel             1048575 non-null  int64  
 1   house.color        1047630 non-null  object 
 2   ni.age             1047573 non-null  float64
 3   len.at.res         1047608 non-null  float64
 4   credit             1047668 non-null  object 
 5   coverage.type      1047595 non-null  object 
 6   dwelling.type      1047582 non-null  object 
 7   premium            1047618 non-null  float64
 8   sales.channel      1047550 non-null  object 
 9   ni.gender          1047615 non-null  object 
 10  ni.marital.status  1047581 non-null  float64
 11  n.adults           1047644 non-null  float64
 12  n.children         1047637 non-null  float64
 13  tenure             1047595 non-null  float64
 14  claim.ind          1047589 non-null  float64
dtype

In [15]:
# missing data percentage list
print("Missing training data:")

total_pct_missing, colnum = 0, 0
for col in train_raw.columns:
    mean_missing = np.mean(train_raw[col].isnull())
    pct_missing = round(mean_missing * 100, 5) # round to 5 decimal places
    total_pct_missing += pct_missing
    
    print("{}. {} - {}%".format(colnum, col, pct_missing))
    colnum += 1

print("\n{}% missing in total\n".format(total_pct_missing))

Missing training data:
0. cancel - 0.0%
1. house.color - 0.09012%
2. ni.age - 0.09556%
3. len.at.res - 0.09222%
4. credit - 0.0865%
5. coverage.type - 0.09346%
6. dwelling.type - 0.0947%
7. premium - 0.09127%
8. sales.channel - 0.09775%
9. ni.gender - 0.09155%
10. ni.marital.status - 0.0948%
11. n.adults - 0.08879%
12. n.children - 0.08945%
13. tenure - 0.09346%
14. claim.ind - 0.09403%

1.2936600000000003% missing in total



In [16]:
# drop observations with over 0 missing values
print("--DROPPING OBSERVATIONS WITH MISSING VALUES--\n")
train_clean = train_raw.dropna()
print(train_clean.shape, "\n")

train_raw_numeric = train_raw.select_dtypes(include=[np.number])
numeric_cols = train_raw_numeric.columns.values

train_clean_non_numeric = train_clean.select_dtypes(exclude=[np.number])
non_numeric_cols = train_clean_non_numeric.columns.values

--DROPPING OBSERVATIONS WITH MISSING VALUES--

(1035101, 15) 



In [17]:
# drop observations with exact same key features
print("--DROPPING DUPLICATE OBSERVATIONS--\n")
key = [feature for feature in numeric_cols if feature != "cancel"] # make sure not to include 'cancel'
train_clean = train_clean.drop_duplicates(subset=key)
print(train_clean.shape, "\n")

--DROPPING DUPLICATE OBSERVATIONS--

(1035101, 15) 



Range Constraints:
- age must be within [18, 100] (tentative)
    + requires graph
- length at residence cannot be negative
    + requires graph
- premium cannot be negative
    + requires graph
- cancel must be within [0, 2]

Membership Constraints:
- not required

Cross-validation Constraints:
- tenure cannot exceed age
- length at residence cannot exceed age
- age and marital status must be legally consistent

Uniqueness Constraints:
- all observations should be unique (tentative)

Data-Type Constraints:
- all features should have consistent data-types

In [21]:
train_clean = train_clean[train_clean['ni.age'] <= 100]
train_clean = train_clean[train_clean['ni.age'] >= 18]
print(train_clean.shape, "\n")

train_clean = train_clean[train_clean['tenure'] < train_clean['ni.age']]
print(train_clean.shape, "\n")

train_clean = train_clean[train_clean['len.at.res'] < train_clean['ni.age']]
print(train_clean.shape, "\n")

train_clean = train_clean[train_clean['cancel'] >= 0]
print(train_clean.shape, "\n")

(1034242, 15) 

(1034242, 15) 

(1028472, 15) 

(1025081, 15) 



In [22]:
# verify no inconsistencies in numerical and categorical values
print("Numeric cols [{}]: {}\n".format(len(numeric_cols), numeric_cols))
for col in numeric_cols:
    print("Categories of {}: {}".format(col, train_clean[col].unique()))
    print(train_clean[col].value_counts(dropna=False), "\n")

print("Non numeric cols [{}]: {}\n".format(len(non_numeric_cols),non_numeric_cols))
for col in non_numeric_cols:
    print("Categories of {}: {}".format(col, train_clean[col].unique()))
    print(train_clean[col].value_counts(dropna=False), "\n")

Numeric cols [9]: ['cancel' 'ni.age' 'len.at.res' 'premium' 'ni.marital.status' 'n.adults'
 'n.children' 'tenure' 'claim.ind']

Categories of cancel: [0 2 1]
0    726606
2    224602
1     73873
Name: cancel, dtype: int64 

Categories of ni.age: [ 37.          40.          45.          36.          51.
  34.          33.          28.          59.          30.
  52.          22.          44.          38.          64.
  49.          54.          46.          70.          60.
  55.          41.          42.          58.          56.
  50.          24.          35.          69.          39.
  57.          43.          61.          63.          31.
  87.          26.          48.          47.          53.
  25.          71.          32.          21.          66.
  23.          65.          27.          62.          80.
  67.          29.          82.          74.          68.
  83.          73.          78.          72.          97.
  98.          20.          77.          81.          85.
 

In [23]:
# make every category a dummy variable
print("--CREATING DUMMY VARIABLES--\n")
train_clean = pd.get_dummies(train_clean, non_numeric_cols)
print(train_clean.shape, "\n")

--CREATING DUMMY VARIABLES--

(1025081, 27) 



In [24]:
# save cleaned df as csv
train_clean.to_csv('data/train_clean.csv')